In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import numpy as np
from fredapi import Fred
import yfinance as yf
import datetime as dt
from tabulate import tabulate
from scipy.stats import norm


In [55]:
#date range 
end_date = dt.datetime.now() 
#look_back_days = 500
#start_date = end_date - dt.timedelta(days= look_back_days)

# #Monte Carlo Data
# simulations = 10000
# confidence_interval = .99
# days = 25


def confidence_interval_and_days():
    # Prompt the user for a confidence interval
    confidence_interval_input = input("Enter the confidence interval (e.g., 0.95 for 95%) or leave it empty for the default (95%): ")
    if confidence_interval_input.strip() == '':
        confidence_interval = 0.95  # Default to 95%
    else:
        confidence_interval = float(confidence_interval_input)

    days_input = input("Enter the number of days or leave it empty for the default (30 days): ")
    if days_input.strip() == '':
        days = 30  # Default to 30 days
    else:
        days = int(days_input)

    return confidence_interval, days
# Example usage:
confidence_interval, days = confidence_interval_and_days()


#Holdings Data
def process_tickers(tickers=None):
    ticker_input = input("Enter a list of tickers separated by spaces (e.g., SPY MSFT TSLA: ")

    default_tickers = ['TLT','BND','LQD','VTI','VOO','VGLT','SRTY','VXX']


    if ticker_input.lower() == 'default' or ticker_input.strip() == '':
        return default_tickers
    else:
        # Check if the input is a single string or a list
        if isinstance(tickers, list):
            return tickers
        else:
            # Split the input string into a list of tickers
            tickers = ticker_input.split()
            return tickers
tickers = process_tickers()
print(tickers)
#weight = [.25,.05,.70]
def portfolio_total_value(port_value=None):
    if isinstance(port_value, int) or isinstance(port_value, float):
        return port_value
    while True: 
        user_input = input("Enter the total Portfolio Value, the system will default to $1,000,000: ")
        if user_input.strip() == '':
            return 1000000
        try:
            port_value = float(user_input)
            return port_value
        except ValueError:
            print("Invalid input. Please enter a valid number or leave it empty.")

    

port_value = portfolio_total_value()

    

def format_pct(var):
    format_pct = "{:}".format(var)
    return format_pct


run_data = [
    ['Days at Risk:', days],
    ['Confidence Interval:', format_pct(confidence_interval)],
    ['Number of Simulations:', simulations]

            ]
model_data = tabulate(run_data, tablefmt='fancy')

def format_number(var):
    formatted_number = "${:,.2f}".format(var)
    return formatted_number
def format_comma(var):
    format_comma = '{:,.0f}'.format(var)
    return format_comma


def start_date(cur_date, years):
    if isinstance(years,int):
        start = cur_date - dt.timedelta(days= years*365)
        return start
    else:
        "The variable years must be an integer"
        
start = start_date(end_date,5)


def port_weights(tickers):
    # Prompt the user to enter weights for each ticker or use default weights
    weights_input = input(f"Enter custom weights separated by spaces, total must be 1 (e.g., 0.4 0.3 0.3), or type 'default' for default equal weights: ")

    if not weights_input.strip() or weights_input.lower() == 'default':
        # If input is empty or 'default' is entered, use default equal weights
        weights = np.array([1/len(tickers)] * len(tickers))
    else:
        # Parse custom weights
        weights = [float(w) for w in weights_input.split() if w.strip()]

        if len(tickers) != len(weights):
            raise ValueError("Number of tickers and custom weights do not match.")

        total_weight = np.sum(weights)
        if total_weight != 1.0:
            raise ValueError("Total weight exceeds 100%. Please ensure the custom weights sum up to 1.")

        weights = np.array(weights)

    # Rest of your code here using the weights

    return weights

def ticker_weights(tickers, weights, total_portfolio_amount):
    if len(tickers) != len(weights):
        raise ValueError("Number of tickers and weights do not match.")

    total_weight = sum(weights)
    if total_weight != 1.0:
        weights = [weight / total_weight for weight in weights]

    ticker_weights_dict = {ticker: weight * total_portfolio_amount for ticker, weight in zip(tickers, weights)}

    return ticker_weights_dict

def validate_tickers(tickers_list):
    valid_tickers = []
    invalid_tickers = []
    
    for ticker in tickers_list:
        try:
            yf.Ticker(ticker)
            valid_tickers.append(ticker)
        except ValueError:
            invalid_tickers.append(ticker)
    
    if invalid_tickers:
        invalid_msg = f"The following tickers are invalid or not found in Yahoo Finance: {', '.join(invalid_tickers)}"
        return invalid_msg
    else:
        invalid_msg = "All tickers are valid and found in Yahoo Finance."
        returns = get_data(valid_tickers, start, end_date)
        return returns

    
    # I think this is worthless. Look to get rid of
def process_stock_tickers(tickers_list=None):
    if tickers_list is None:
        return "Please enter a list of tickers into the function process_stock_tickers."

    invalid_tickers_msg = validate_tickers(tickers_list)
    if invalid_tickers_msg:
        return invalid_tickers_msg


def get_data(tickers, startd, endd):
    returns = pd.DataFrame() 
    for ticker in tickers:
        data = yf.download(ticker, start=startd, end=endd) 
        
        returns.loc[:, ticker] = data['Adj Close']
        
    l_returns = np.log(returns / returns.shift(1))
    l_returns = l_returns.dropna()
    return l_returns
     

### daily returns output 
def expected_return(weights, l_return):
    expected = np.sum(l_return.mean()*weights)
    return expected

def standard_dev(weights, cov):
    ### T transposes the data
    variance = weights.T @ cov@weights
    return np.sqrt(variance)

def z_score():
    z_score = np.random.normal(0,1)
    return z_score


def agg_fun(l):
    cov = l_returns.cov()
     
    port_expected_return = expected_return(weights, l_returns)
    port_std = standard_dev(weights, cov)
    return port_expected_return, port_std


def scenario(port_value, z_score, port_std, days):
    gainloss = port_value * port_expected_return * days + port_value * port_std * z_score * np.sqrt(days)
    return gainloss

def scenario_run(simulations):
    scenario_result = []
    for i in range(simulations):
        z = z_score()
        scenario_result.append(scenario(port_value, z, port_std, days))
    VaR = np.percentile(scenario_result, 100*(1- confidence_interval))
    
    return format_number(VaR)


#### Change weights here ###
weights = port_weights(tickers)

l_returns = validate_tickers(tickers)

port_expected_return, port_std = agg_fun(l_returns)

final = scenario_run(simulations)



data_tick = {"Tickers":tickers,"Port Weight":weights}
df_tick_weight = pd.DataFrame(data_tick)


final_display = {
    'Value at Risk': final,
    'Days': days,
    'Portfolio Value': format_number(port_value),
    'Confidence Interval': format_pct(confidence_interval),
    'Number of Simulations': format_comma(simulations),
    'Number of Positions': len(tickers)
}
final_table = [[key, value] for key, value in final_display.items()]
final_tab = tabulate(final_table,tablefmt='fancy')


print(final_tab)

port_breakdown = tabulate(df_tick_weight, headers='keys', tablefmt='fancy',showindex=False)
print("\n" + port_breakdown)

Enter the confidence interval (e.g., 0.95 for 95%) or leave it empty for the default (95%):  
Enter the number of days or leave it empty for the default (30 days):  
Enter a list of tickers separated by spaces (e.g., SPY MSFT TSLA:  


['TLT', 'BND', 'LQD', 'VTI', 'VOO', 'VGLT', 'SRTY', 'VXX']


Enter the total Portfolio Value, the system will default to $1,000,000:  
Enter custom weights separated by spaces, total must be 1 (e.g., 0.4 0.3 0.3), or type 'default' for default equal weights:  


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
---------------------  -------------
Value at Risk          $-93,874.46
Days                   30
Portfolio Value        $1,000,000.00
Confidence Interval    0.95
Number of Simulations  10,000
Number of Positions    8
---------------------  -------------

Tickers      Port Weight
---------  -------------
TLT                0.125
BND                0.125
LQD                0.125
VTI                0.125
VOO                0.125
VGLT             